In [68]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import requests
import time
import json

In [69]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [70]:
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [71]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [72]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[{'_id': ObjectId('675232ad2766f7dfb642df3d'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 0, 9, 33, 207000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('675225de2766f7dfb642df3b'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 5, 23, 14, 54, 932000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6752248c2766f7dfb642df39'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 5, 23, 9, 16, 152000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('675224312766f7dfb642df37'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 5, 23, 7, 45, 834000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6752241f2766f7dfb642df35'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 5, 23, 7, 27, 201000), 'content': 'I 

In [73]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        self.opinions = {}
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input}) (Opinions: {self.opinions})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender

In [74]:
Sami = Agent("Sami", "agent_001", "Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has an opinion on everything and is kind most of the time.", "male")

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is the sweetest person on earth and would never lie or attack someone.", "female")

Sami.opinions["Eliana"] = "He likes Eliana and thinks her cooking advices are great"
Eliana.opinions["Sami"] = "She thinks Sami is cool and she likes his cooking"

print(Sami)
print(Eliana)

Agent Sami (ID: agent_001) (Description: Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has an opinion on everything and is kind most of the time.) (Opinions: {'Eliana': 'He likes Eliana and thinks her cooking advices are great'})
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is the sweetest person on earth and would never lie or attack someone.) (Opinions: {'Sami': 'She thinks Sami is cool and she likes his cooking'})


In [79]:
def make_initial_prompt(agent1, agent2, subject):
    message_content = f"""
            Context:
            Here is a description of {agent1.name}: {agent1.user_input}.
            Here is a description of {agent2.name}: {agent2.user_input}.
            Here is what {agent1.name} thinks about {agent2.name}:
            {agent1.opinions[agent2.name]}
            Here is what {agent2.name} thinks about {agent1.name}:
            {agent2.opinions[agent1.name]}
            The context of this conversation is: {subject}
            Start directly with the conversation, not a introduction phrase. Create the conversation they had:

            {agent1.name}: "
        """
    return message_content

def make_subject(memory):
    message_content = f"""
            Context:
            This is the memory of a conversation between 2 people:
            {memory}
            Make a summary in 1 sentence or 2 with the important points and details of the conversation (for example, if they are talking about doing an activity or a dish together, keep it in memory), ignorig the first sentence.
            Start directly with the summary, not with a phrase like "here is a summary".
        """
    return message_content

def extract_emotion(memory, agent_concerned, other):
    message_content = f"""
    Agent1: {agent_concerned.name}, {agent_concerned.gender}, {agent_concerned.user_input}
    Agent2: {other.name}, {other.gender}, {other.user_input}
    Conversation: "{memory}"
    Possible emotions: ["happy", "sad", "angry", "neutral", "fearful"]

    Analyze the information about the agents and their conversation, and identify the primary emotion felt by Agent1 as a result of this interaction.
    The response MUST be a SINGLE word from the provided list of emotions, only from the provided list, nothing more.
    """
    return message_content

def update_opinion(memory, agent_concerned, other):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Conversation: "{memory}"

    Here is the conversation that happened between {agent_concerned.name} and {other.name}.
    Summarize what {agent_concerned.name} thought about {other.name} in one short sentence. The sentence needs to be in third person:
    """
    return message_content

def update_opinion2(opinion, agent_concerned, other, emotion=None):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Last opinion of {agent_concerned.name} about {other.name}: {agent_concerned.opinions[other.name]}
    Analyzed phrase or conversation: "{opinion}"
    {f'Emotion felt by {agent_concerned.name}: "{emotion}"' if emotion else ''}

    Based on the information about both agents and the provided text, update the opinion of {agent_concerned.name} about {other.name}.
    If an emotion is specified, take it into account to adjust the tone or content of this opinion.
    If a part of the old opinion is correct, keep it in the new opinion.
    Juste give the new opinion, nothing else.
    """
    return message_content

In [76]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agent1, agent2, subject, use_memory = True):
    # First agent's turn
    memories = temp_retrieve_memories(agent1.agent_id)
    if (len(memories) > 0 and use_memory):
        subject_prompt = make_subject(memories[0])
        data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": subject_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }
        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
        subject = "Here is the resume of the past conversation: " + response + " They meet some time later"
        print("Subject :", subject)
    
    message_prompt = make_initial_prompt(agent1, agent2, subject)

    data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
    print(response)
    #temp_store_memory(agent1.agent_id, [agent2.agent_id], str(response))
    #temp_store_memory(agent2.agent_id, [agent1.agent_id], str(response))
    return response

In [81]:
dialog = chat(Sami, Eliana, "They are having an argument about lasagna", False)

Sami: "Come on Eliana, you can't be serious about adding more than three types of cheese to my lasagna recipe! I know you think you're a spice expert or something, but trust me, it's perfect just the way it is."

Eliana: "Oh Sami, don't be so stubborn! I'm telling you, that extra parmesan and gorgonzola will take the lasagna to a whole new level. You can't put too much of a good thing on pasta!"

Sami: "But what about the balance? The flavors are already perfect with just two types of cheese. Adding more than that would throw everything off."

Eliana: "Balance is overrated! A little extra richness never hurt anyone. And besides, I have a special blend of spices that will complement the cheese perfectly. You can't just rely on your Italian grandmother's recipe without trying something new and exciting."

Sami: "Exciting? This is Italy, Eliana! We don't mess with traditional recipes like that. Besides, I'm pretty sure Nonna would turn over in her grave if she knew you were trying to mode

In [83]:
subject_prompt = extract_emotion(dialog, Sami, Eliana)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
emotion = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("emotion:", emotion)

subject_prompt = update_opinion(dialog, agent_concerned=Sami, other=Eliana)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
opinion = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("\nOpinion about the conversation:", opinion)

subject_prompt = update_opinion2(opinion, agent_concerned=Sami, other=Eliana, emotion=emotion)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("\nnew opinion:", response)

emotion: Angry

Opinion about the conversation: Sami had a loving but slightly exasperated attitude towards Eliana, appreciating her enthusiasm for food and spices while also being skeptical of her modernizing tendencies.

new opinion: Sami has a loving but slightly exasperated attitude towards Eliana, appreciating her enthusiasm for food and spices while also being frustrated with her tendency to modernize traditional recipes too much.
